In [34]:
import os
from pathlib import Path

In [35]:
%pwd

'/home/black/Desktop/Nano Gpt/prepare'

In [36]:
# We always start with a dataset to train on. Let's download the tiny shakespeare dataset

if not os.path.exists('input.txt'):
    !wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
else:
    with open('input.txt', 'r', encoding='utf-8') as f:
        text = f.read()

In [37]:
print("Length of dataset in charqcters: ", len(text))

Length of dataset in charqcters:  1115394


In [38]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [39]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [40]:
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }

encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])


print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [41]:
import torch

data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])



torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [42]:
#  Split data into tran and validataion data

n = int(0.9*len(data))
train_data = data[:n]   
val_data = data[n:]

In [43]:
block_size = 11

train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43])

In [44]:
x = train_data[:block_size]
y = train_data[:block_size+1]

for i in range(block_size):
    context = x[:i+1]
    target = y[i]
    print(f"when input is {context} the target: {target}")

when input is tensor([18]) the target: 18
when input is tensor([18, 47]) the target: 47
when input is tensor([18, 47, 56]) the target: 56
when input is tensor([18, 47, 56, 57]) the target: 57
when input is tensor([18, 47, 56, 57, 58]) the target: 58
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47, 58]) the target: 58
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64]) the target: 64


In [45]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8


def get_batch(split):
    data = train_data if split == 'train' else val_data

    # torch.randint(a, b, size) ---> torch.randint(len(data) - block_size, (batch_size,)) 

    ix = torch.randint(len(data) - block_size, (batch_size,))  
    print("iX: ",ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y



xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')


for i in range(batch_size):
    for j in range(block_size):
        context = xb[i,:j+1]
        target = yb[i,j]
        print(f"when input is {context.tolist()} the target: {target}")

iX:  tensor([ 76049, 234249, 934904, 560986])
inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
----
when input is [24] the target: 43
when input is [24, 43] the target: 58
when input is [24, 43, 58] the target: 5
when input is [24, 43, 58, 5] the target: 57
when input is [24, 43, 58, 5, 57] the target: 1
when input is [24, 43, 58, 5, 57, 1] the target: 46
when input is [24, 43, 58, 5, 57, 1, 46] the target: 43
when input is [24, 43, 58, 5, 57, 1, 46, 43] the target: 39
when input is [44] the target: 53
when input is [44, 53] the target: 56
when input is [44, 53, 56] the target: 1
when input is [44, 53, 56, 1] the target: 58
when input is [44, 53, 5

In [46]:
print(xb)

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])


In [47]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self,vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size,  vocab_size)

    ''' it's the forward function that defines the network structure
        we're accepting only a single input in here, but if you want'''   

    def forward(self, idx, targets= None):
        logits = self.token_embedding_table(idx) # (B,T,C)
        


        #NLLLOSS : negative log likelihood loss. It is best suited for classification problems.
        #This is particularly useful when you have an unbalanced training set.


        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [48]:
#Optimiser
optimizer = torch.optim.Adam(m.parameters(), lr=1e-3)

In [49]:
batch_size = 32

for steps in range(100):
    xb, yb = get_batch('train')

    logits, loss = m(xb, yb)

    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())


iX:  tensor([880792, 620608, 758123, 761955, 483803,  61808, 476059, 726263, 656633,
        674426, 241399, 395150, 298543, 938016, 254737, 952644,  24901, 772494,
        628396, 556307, 319255,  38208, 849969, 394140, 919551, 316361, 359631,
        121698, 567966, 743553, 884616, 849510])
iX:  tensor([793375, 924645,  86369, 919544, 299140, 227344, 860300, 216165, 551383,
        555439, 641236, 435602, 759848, 700082, 918994, 468585, 273700,  95491,
        765171, 359861, 736914, 144022, 105516, 477200, 784451, 896134, 934681,
        376322, 502277, 204528, 202156, 543499])
iX:  tensor([ 800823,  930616,  153375,  834582,  860265,  296227,  993134,  806983,
          98006,  172503,  194830,  353142,  954475,  301736,  781978, 1002223,
         143971,  716105,  174172,  148687,  362485,  354704,  925089,  290609,
         108944,  926805,  989259,  608356,  416002,  157300,  833640,  657117])
iX:  tensor([680430, 266960, 331912, 406615, 677171,  61696, 506942, 766551, 981292,
 

In [57]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


SAr QJaKMlgBSGGSBDXIynAmD!zt;,!wfavrwQJqMOcQbOrJFtCAU:.DzPmIBCMphs
jUvvkth.S $I&vxmfaSP'HT'nxkhzoixGtw pMgYI&dYXyJ,iBU!upJapNl!.A!BPJ?YA:y,Sjv.SP,y'DDuwfRj;qZcqhepxEZ-IVcoteAajxa!W?pQOdIkQT:CBKM&SSl?s-!Pmh.. AcfNmviBUinQh$GUqHdmgCjyMfYw,SVY!ugd!,SObQ&!pcxFbyfR.pDAQbmuOM&yh$zL&u!BFZGNMA:fMkHxKuQ!K vnQJlEUzryI&GlysmCmLBy.DPER:L;vehRJBjNMNWWP,Xyxr'P!PAdbrbnASVBkRp:LbHWz?YC ,evN:EH?G&vPLQ!mChs,lnWbjA-k.uFZdhEHD,&ula;vT3I!UzWN!$RrgOr'R
,.be,RdZsxToNtL,
REsrJBOdxJAY!.Z;.NwNvoTxERj?w$XZ?3KH-oRIKpDev!3!


# The mathematical trick in self-attention

In [58]:
torch.manual_seed(1337)

a = torch.tril(torch.ones(3,3))
a = a/ torch.sum(a, dim=1, keepdim=True)
b = torch.randint(0, 10, (3, 2)).float()
c = a @ b

print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)


a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=
tensor([[5., 7.],
        [2., 0.],
        [5., 3.]])
--
c=
tensor([[5.0000, 7.0000],
        [3.5000, 3.5000],
        [4.0000, 3.3333]])
